##### Besluit verbeterpuntjes:
- Als besluit als eerste aan de pagina staat pakt hij het niet door de ". Besluit"
- Soms heet het kopje 'Beslissing' of 'Beslissing op uw bezwaar', 'Besluit op bezwaar' door de tweede ziet hij geen hoofdletter na ". Besluit X"
- Soms wordt de voorgaande zin niet correct afgesloten met een punt, hierdoor herkent de regex het niet. 
- Soms eindigt de zin met een hoofdletter. Voorbeeld: 
    - "separaat bericht. 3597201-1048329-WJZ Besluit Ik bestuit de door u gevraagde informatie nagenoeg geheel openbaar te maken." (nl.mnre1025.2i.2023.74.doc.1)
- Soms heet het kopje 'Deelbesluit' of 'Conclusie'
- 1x gezien dat er helemaal geen kopje Besluit staat in wat voor vorm dan ook, besluit staat in de lopende tekst
- Besluit ziet er zo uit: 'B 1 it' OCR foutje

### Spacy modellen:
https://drive.google.com/drive/folders/1AzG_0rQbi4SYRyxSno41R8wCUszkRG-r?usp=sharing

In [2]:
# Imports
import re
import spacy
import string
import json
import numpy as np
import pandas as pd
from spacy import displacy
from spacy.tokens import DocBin
import dateparser
from datetime import datetime, timezone, timedelta
import pickle
from IPython.display import HTML

pd.options.mode.chained_assignment = None

In [3]:
def html(df):
    display(HTML(df.to_html(render_links=True, escape=False)))

In [4]:
min2023 = pd.read_pickle("../pipe/data/minfinal.pkl.gz", compression='gzip')

# een pagina per document, met daaronder hoeveel wel, en hoeveel niet worden opgehaald

min2023_1 = min2023[min2023.index.get_level_values(1) == 1]
len(min2023_1[~min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']]), len(min2023_1[min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']])

C:\Users\Joran\AppData\Local\Temp\ipykernel_10612\561825764.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(min2023_1[~min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']]), len(min2023_1[min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']])
C:\Users\Joran\AppData\Local\Temp\ipykernel_10612\561825764.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(min2023_1[~min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']]), len(min2023_1[min2023['besluit_context'].isna()][['besluit_context', 'foi_bodyTextOCR', 'dc_source']])


(781, 178)

In [5]:
# model = 'nl_besluit_902_lg'
# model = 'nl_besluit_1000_lg'
model = 'nl_besluit_gescheiden_lg'
# model = 'nl_besluit_all_lg'

nlp = spacy.load("T:/Desktop/UvA/Jaar3/OndragelijkRepo/noclone/Notebooks/Experimenten/data/spacy_custom_models/" + model)

def type_besluit(besluit_text):
    if besluit_text != None:
        doc = nlp(besluit_text)
        hoogste = max(doc.cats, key=doc.cats.get)
        perc = doc.cats[hoogste]
        return hoogste, round(perc*100, 2)
    else: 
        return None, None
    
min2023_1['besluit_spacy'] = min2023_1['besluit_context'].apply(lambda x: type_besluit(x))
min2023_1[['besluit_type', 'besluit_zekerheid']] = pd.DataFrame(min2023_1['besluit_spacy'].tolist(), index=min2023_1.index)
min2023_1.drop('besluit_spacy', axis=1, inplace=True)

In [6]:
min2023_1[['besluit_type', 'besluit_zekerheid']].groupby('besluit_type').describe()

besluit_zekerheid                                               \
                           count       mean        std    min      25%    50%   
besluit_type                                                                    
afgewezen                  109.0  96.452569   9.606012  44.78  98.7200  99.83   
deels openbaar             519.0  99.062331   4.066234  55.73  99.3900  99.94   
niet aanwezig               88.0  97.755568   8.421538  50.79  99.8925  99.99   
openbaar                    65.0  93.952769  10.169911  59.14  94.3300  99.21   

                                
                   75%     max  
besluit_type                    
afgewezen        99.96  100.00  
deels openbaar   99.99  100.00  
niet aanwezig   100.00  100.00  
openbaar         99.84   99.94

In [28]:
html(min2023_1[['besluit_type', 'besluit_zekerheid', 'besluit_context']].sample(n=15))

,,besluit_type,besluit_zekerheid,besluit_context
foi_documentId,foi_pageNumber,,,
nl.mnre1153.2i.2023.642.doc.1,1,deels openbaar,99.39,Ik besluit een deel van de door u gevraagde informatie niet openbaar te maken.
nl.mnre1025.2i.2023.16.doc.1,1,deels openbaar,100.00,"Ik besluit de door u gevraagde informatie grotendeels openbaar te maken. Ik licht mijn besluit hieronder toe. Inventarisatie documenten Op basis van uw verzoek zijn meerdere documenten aangetroffen. Deze documenten zijn opgenomen in een inventarislijst, zie hiertoe bijlage B."
nl.mnre1153.2i.2023.35.besluit.1,1,niet aanwezig,97.28,"De gevraagde informatie berust niet bij RVO. Om deze reden wordt uw verzoek afgewezen. Toelichting De informatie is beschikbaar op de website van de Stichting Controle Orgaan Kwaliteitszaken (COKZ). Deze is toezichthouder op de veiligheid en kwaliteit van in Nederland geproduceerde zuivelproducten, eieren en eiproducten."
nl.mnre1058.2i.2023.268.doc.1,1,deels openbaar,99.99,Gedeeltelijk openbaar Ik besluit om 5 documenten gedeeltelijk openbaar te maken van de door u opgevraagde informatie. Op de inventarislijst is er per document aangegeven welk uitzonderingsgrond van toepassing is. Geheel openbaar Ik besluit 2 documenten geheel openbaar te maken van de door u opgevraagde informatie. Dit houdt in dat ik geen weigeringsgronden heb toegepast op het te verstrekken document.
nl.mnre1090.2i.2023.28.besluit.1,1,afgewezen,99.98,Ik kan uw verzoek niet inwilligen. De Woo is niet van toepassing op reeds openbare informatie. Ik wijs uw verzoek om informatie derhalve af. Hiertoe merk ik het volgende op.
nl.mnre1045.2i.2023.96.doc.1,1,deels openbaar,99.39,Ik besluit een deel van de door u gevraagde informatie niet openbaar te maken.
nl.mnre1153.2i.2023.669.doc.1,1,deels openbaar,99.94,"Ik heb besloten de informatie die u vraagt gedeeltelijk openbaar te maken. Waarom ik een deel van de informatie niet openbaar maak, leest u verderop in deze brief. In bijlage 1 vindt u een opsomming van de artikelen uit de Woo die voor mijn besluit van belang zijn. Beoordeling van uw verzoek Uw verzoek gaat over publieke informatie die in één of meer documenten staat."
nl.mnre1090.2i.2023.2.besluit.1,1,deels openbaar,99.99,Ik besluit de door u gevraagde informatie gedeeltelijk openbaar te maken. Per document is op de inventarislijst aangegeven welke uitzonderingsgronden zijn toegepast. De inventarislijst maakt integraal onderdeel uit van het besluit. Voor de motivering van dit besluit verwijs ik naar het onderdeel “Overwegingen” van deze brief.
nl.mnre1153.2i.2023.363.doc.1,1,deels openbaar,99.94,"Ik besluit het document openbaar te maken, met uitzondering van de persoonsgegevens die daarin staan."
